## ETL Process for TV Attribution Case

### Script for table creations in sqlite3

#### Analyst: Jose David Perez - 22/06/2021

#### Libraries needed for data manipulation

In [1]:
import pandas as pd
import sqlite3 as sq3
import numpy as np
from datetime import datetime

#### Accesing the data

In [2]:
# Reading sessions.csv
custom_date_parser = lambda x: datetime.strptime(x, "%d.%m.%y %H:%M")

sessions = pd.read_csv('case_may_2020_sessions_final.csv', usecols=[1,2,3,4], parse_dates=['date_hour_minute'], date_parser=custom_date_parser);
sessions.head(1)

,origin,channel,identifier,date_hour_minute
0,app,Direct App,13dd4f3336931ae1e9ad25cdd8,2020-05-15


In [3]:
# Reading audience sheet
audience = pd.read_excel('case_may_2020_TV data.xlsx', sheet_name='Audience', usecols=np.arange(3))
audience = audience.reindex(columns=['tv_show_id', 'tv_show', 'reach'])
audience.head()

,tv_show_id,tv_show,reach
0,1,First Dates - Ein Tisch für zwei,57000
1,2,SOKO Kitzbühel,95000
2,3,Alles was zählt,86000
3,4,Jack Reacher,162000
4,5,Der Bulle von Tölz: Tod aus dem All,150000


In [4]:
# Reading tv planning sheet
tv_planning = pd.read_excel('case_may_2020_TV data.xlsx', sheet_name='TV Planning', usecols=np.arange(7))
tv_planning.head()

,tvp_id,block_start_time,tv_show_id,tv_show,block_size,block_position,spot_id
0,10001127236681063068_1553644648,2020-05-01 18:15:00.000,1,First Dates - Ein Tisch für zwei,420,345,9122
1,10001127236681063068_1553644679,2020-05-01 19:36:05.000,3,Alles was zählt,60,15,6561
2,10001127236681063068_1553644709,2020-05-01 20:20:00.000,5,Der Bulle von Tölz: Tod aus dem All,480,60,4242
3,10001127236681063068_1553644649,2020-05-02 18:14:59.995,6,Newstime,300,270,1234
4,10001127236681063068_1553644680,2020-05-02 19:36:04.995,7,Comeback oder weg?,420,345,4234


In [5]:
# Reading tv spots sheet
tv_spots = pd.read_excel('case_may_2020_TV data.xlsx', sheet_name='TV spots', usecols=np.arange(3))
tv_spots.head()

,spot_id,spot_name,duration
0,4234,springsummer_branding,30 secs
1,6232,springsummer_outfitfashionA,15 secs
2,7224,springsummer_saleA,7 secs
3,8331,springsummer_saleB,7 secs
4,9122,springsummer_saleC,7 secs


In [6]:
# Reading tv visits sheet
tv_visits = pd.read_excel('case_may_2020_TV data.xlsx', sheet_name='TV - visits match', usecols=np.arange(7))
tv_visits.head()

,origin,channel,identifier,date_hour_minute,tvp_id,tv_probability,spot_weight
0,app,Direct App,68308b2ca17ca3554546bbf7e8,2020-05-15 18:10:00,10001127236681063068_1553644644,0.288847,1.0
1,app,Direct App,cb51890263cb49761253403353,2020-05-15 18:10:00,10001127236681063068_1553644644,0.288847,1.0
2,app,Direct App,cca4b65d88d67b1acf79c4dc71,2020-05-15 18:10:00,10001127236681063068_1553644644,0.288847,1.0
3,web,Direct,234371511550390009,2020-05-15 18:10:00,10001127236681063068_1553644644,0.288847,1.0
4,web,Direct,5798477480565300009,2020-05-15 18:10:00,10001127236681063068_1553644644,0.288847,1.0


#### Getting meta data

In [7]:
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058463 entries, 0 to 1058462
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   origin            1058463 non-null  object        
 1   channel           1058463 non-null  object        
 2   identifier        1058463 non-null  object        
 3   date_hour_minute  1058463 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 32.3+ MB


In [8]:
audience.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tv_show_id  20 non-null     int64 
 1   tv_show     20 non-null     object
 2   reach       20 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 608.0+ bytes


In [9]:
tv_planning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tvp_id            95 non-null     object        
 1   block_start_time  95 non-null     datetime64[ns]
 2   tv_show_id        95 non-null     int64         
 3   tv_show           95 non-null     object        
 4   block_size        95 non-null     int64         
 5   block_position    95 non-null     int64         
 6   spot_id           95 non-null     int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 5.3+ KB


In [10]:
tv_spots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   spot_id    9 non-null      int64 
 1   spot_name  9 non-null      object
 2   duration   9 non-null      object
dtypes: int64(1), object(2)
memory usage: 344.0+ bytes


In [11]:
tv_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18662 entries, 0 to 18661
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   origin            18662 non-null  object        
 1   channel           18662 non-null  object        
 2   identifier        18662 non-null  object        
 3   date_hour_minute  18662 non-null  datetime64[ns]
 4   tvp_id            18662 non-null  object        
 5   tv_probability    18662 non-null  float64       
 6   spot_weight       18662 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 1020.7+ KB


#### Cleaning and transforming features

In [12]:
# Converting date_hour_minute to string for sqlite3
sessions['date_hour_minute'] = sessions['date_hour_minute'].astype(str)
sessions.head(1)

,origin,channel,identifier,date_hour_minute
0,app,Direct App,13dd4f3336931ae1e9ad25cdd8,2020-05-15 00:00:00


In [13]:
# Dropping tv_show from tv_planning
tv_planning.drop(columns=['tv_show'], inplace=True)
tv_planning.head()

,tvp_id,block_start_time,tv_show_id,block_size,block_position,spot_id
0,10001127236681063068_1553644648,2020-05-01 18:15:00.000,1,420,345,9122
1,10001127236681063068_1553644679,2020-05-01 19:36:05.000,3,60,15,6561
2,10001127236681063068_1553644709,2020-05-01 20:20:00.000,5,480,60,4242
3,10001127236681063068_1553644649,2020-05-02 18:14:59.995,6,300,270,1234
4,10001127236681063068_1553644680,2020-05-02 19:36:04.995,7,420,345,4234


In [14]:
# Converting block_start_time to string for sqlite3
tv_planning['block_start_time'] = tv_planning['block_start_time'].astype('str')
tv_planning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tvp_id            95 non-null     object
 1   block_start_time  95 non-null     object
 2   tv_show_id        95 non-null     int64 
 3   block_size        95 non-null     int64 
 4   block_position    95 non-null     int64 
 5   spot_id           95 non-null     int64 
dtypes: int64(4), object(2)
memory usage: 4.6+ KB


In [15]:
# Renaming duration on tv_spots and removing 'secs'
tv_spots.rename(columns={'duration': 'duration_secs'}, inplace=True)

tv_spots['duration_secs'] = pd.to_numeric(tv_spots['duration_secs'].astype('str').str[:-5])
tv_spots['duration_secs']

0    30
1    15
2     7
3     7
4     7
5    30
6     7
7     7
8    30
Name: duration_secs, dtype: int64

#### Database and table creation

In [16]:
# Creating database on sqlite3
conn = sq3.connect('tv_attribution_case')

c = conn.cursor()

In [17]:
# Executing DDL por table creation
c.execute('''CREATE TABLE sessions
            ([origin] text, [channel] text, [identifier] text, [date_hour_minute] text)''')

c.execute('''CREATE TABLE audience
            ([tv_show_id] INTEGER PRIMARY KEY, [tv_show] text, [reach] INTEGER)''')

c.execute('''CREATE TABLE tv_planning
            ([tvp_id] text PRIMARY KEY, [block_start_time] text, [tv_show_id] INTEGER, [block_size] INTEGER, 
            [block_position] INTEGER, [spot_id] INTEGER)''')

c.execute('''CREATE TABLE tv_spots
            ([spot_id] INTEGER PRIMARY KEY, [spot_name] text, [duration_secs] INTEGER)''')

c.execute('''CREATE TABLE tv_visits
            ([origin] text, [channel] text, [identifier] text, [date_hour_minute] text, [tvp_id] text, 
            [tv_probability] REAL, [spot_weight] REAL)''')

In [18]:
# Inserting rows from pandas dataframe to sqlite
sessions.to_sql('sessions', conn, if_exists='append', index=False)
audience.to_sql('audience', conn, if_exists='append', index=False)
tv_planning.to_sql('tv_planning', conn, if_exists='append', index=False)
tv_spots.to_sql('tv_spots', conn, if_exists='append', index=False)
tv_visits.to_sql('tv_visits', conn, if_exists='append', index=False)

In [19]:
# Commit changes and close database
conn.commit()
conn.close()